# Pistachio simulation

<!-- Index of contents -->
* [Define the problem](#Define-the-problem)
* [Simulation agents](#Simulation-agents)
* [Stress types](#Stress-types)
  * [Chill hours stress](#Chill-hours-stress)
  * [Heat units stress](#Heat-units-stress)
  * [Temperature stress](#Temperature-stress)
  * [Hydric stress](#Hydric-stress)
* [Simulation parameters](#Simulation-parameters)

## Define the problem

......

## Simulation agents

In [66]:
from pydantic import BaseModel, Field, field_validator, model_validator
from typing import List, Literal, Optional
from datetime import date
import numpy as np
from typing import Tuple

### Geolocation data

In [67]:
# Meteorological Class
class Meteorological(BaseModel):
    timestamp: date = Field(..., description="Date of the meteorological data")
    temperature: float = Field(..., ge=-30, le=50, description="Current temperature [-30°C to 50°C]")
    relative_humidity: float = Field(..., ge=0, le=100, description="Current relative humidity [0-100%]")
    wind: float = Field(..., ge=0, le=100, description="Current wind speed [0-100 m/s]")
    precipitation: float = Field(..., ge=0, le=100, description="Precipitation level [0-100 mm]")

# Weather Class
class Weather(BaseModel):
    type: Literal["normal", "extreme"] = Field(..., description="Type of weather [normal, extreme]")
    meteorological: List[Meteorological] = Field(..., description="List of meteorological data")

    def __init__(self):
        self.__transform_meteorological_data()

    def __transform_meteorological_data(self):
        match self.type:
            case "extreme":
                # NOTE: This is a dummy example, in a real case
                for meteorological in self.meteorological:
                    meteorological.temperature += np.random.choice([-5, 5])
                    meteorological.relative_humidity += np.random.choice([-10, 10])
                    meteorological.wind += np.random.choice([-10, 10])
                    meteorological.precipitation += np.random.choice([-10, 10])

# Geolocation Class
class Geolocation(BaseModel):
    location: str = Field(..., description="Geographic location, e.g., Valladolid")
    date_init: date = Field(..., description="Initial date")
    date_end: date = Field(..., description="End date")
    weather: Weather

    # On init, call the API and get the weather data depending on the location and dates
    def __init__(self, location, date_init, date_end):
        self.location = location
        self.date_init = date_init
        self.date_end = date_end
        self.weather = self.__get_weather_data()

    def __get_weather_data(self):
        # TODO: Call the API and get the weather data
        pass

### Disease agents

<style scoped>
table {
  font-size: 16px;
  font-family: Verdana, sans-serif;
}
</style>

| Disease          | Temperature       | Humidity     | Severity   | Treatment difficulty | Period               |
|------------------|-------------------|--------------|------------|-----------------------|----------------------|
| Verticillium     | $20 - 30^{\circ}C$ | Better high  | Very high  | Very high            | June - September     |
| Botryosphaeria   | $27 - 33^{\circ}C$ | Better high  | High       | Medium               | June - September     |
| Alternaria       | $27 - 35^{\circ}C$ | Better high  | Medium     | Medium               | mid July - September |
| Septoria         | $18 - 26^{\circ}C$ | Better high  | High       | Medium               | May - September      |
| Aflatoxins       | $27 - 40^{\circ}C$ | Better high  | Very High  | High                 | Mid August - Harvest |


In [68]:
from enum import Enum

class DiseaseType(str, Enum):
    Verticillium = "Verticillium"
    Botryosphaeria = "Botryosphaeria"
    Alternaria = "Alternaria"
    Septoria = "Septoria"
    Aflatoxin = "Aflatoxin"

class Disease(BaseModel):
    type: DiseaseType = Field(..., description="Type of disease, e.g., Verticillium, Alternaria")
    RH_disease_min: float = Field(default=65, ge=0, le=100, description="Minimum relative humidity for disease")
    T_disease_min: float = Field(..., ge=0, le=50, description="Minimum temperature for disease")
    T_disease_max: float = Field(..., ge=0, le=50, description="Maximum temperature for disease")
    severity: float = Field(..., ge=0, le=1.0, description="Severity of the disease [low=0.0, high=1.0]")
    treatment_difficulty: float = Field(..., ge=0, le=1.0, description="Difficulty to treat [low=0.0, high=1.0]")
    period_init: int = Field(..., ge=0, le=365, description="Month of the year when the disease can appear")
    period_end: int = Field(..., ge=0, le=365, description="Month of the year when the disease disappears")

    # Allow arbitrary types (ndarray in this case)
    class Config:
        arbitrary_types_allowed = True

    @field_validator('period_end')
    def check_period(cls, period_end, values):
        if period_end < values['period_init']:
            raise ValueError('period_end must be greater than period_init')
        return period_end
        

# Verticillium Class
class Verticillium(Disease):
    type: DiseaseType = DiseaseType.Verticillium
    T_disease_min: float = 20
    T_disease_max: float = 30
    severity: float = np.random.uniform(0.8, 0.9)
    treatment_difficulty: float = np.random.uniform(0.8, 0.9)
    period_init: int = 6 # June
    period_end: int = 9 # September

# Botryosphaeria Class
class Botryosphaeria(Disease):
    type: DiseaseType = DiseaseType.Botryosphaeria
    T_disease_min: float = 27
    T_disease_max: float = 33
    severity: float = np.random.uniform(0.5, 0.7)
    treatment_difficulty: float = np.random.uniform(0.3, 0.6)
    period_init: int = 6 # June
    period_end: int = 9 # September

# Alternaria Class
class Alternaria(Disease):
    type: DiseaseType = DiseaseType.Alternaria
    T_disease_min: float = 27
    T_disease_max: float = 35
    severity: float = np.random.uniform(0.3, 0.6)
    treatment_difficulty: float = np.random.uniform(0.3, 0.6)
    period_init: int = 7 # July
    period_end: int = 9 # September

# Septoria Class
class Septoria(Disease):
    type: DiseaseType = DiseaseType.Septoria
    T_disease_min: float = 18
    T_disease_max: float = 26
    severity: float = np.random.uniform(0.5, 0.7)
    treatment_difficulty: float = np.random.uniform(0.3, 0.6)
    period_init: int = 5 # May
    period_end: int = 9 # September

# Aflatoxin Class
class Aflatoxin(Disease):
    type: DiseaseType = DiseaseType.Aflatoxin
    T_disease_min: float = 25
    T_disease_max: float = 35
    severity: float = np.random.uniform(0.8, 0.95)
    treatment_difficulty: float = np.random.uniform(0.6, 0.8)
    period_init: int = 8 # August
    period_end: int = 10 # October

### Pest agents

<style scoped>
table {
  font-size: 16px;
  font-family: Verdana, sans-serif;
}
</style>

| Pest               | Temperature       | Humidity     | Severity | Treatment difficulty | Period             |
|--------------------|-------------------|--------------|----------|-----------------------|--------------------|
| Green stink bug    | $20 - 35^{\circ}C$ | Better high  | Medium   | High                 | August - October   |
| Pistachio psylla   | $20 - 35^{\circ}C$ | Better high  | High     | High                 | April - October    |
| Leaf beetle        | $20 - 35^{\circ}C$ | Better high  | Low      | Medium               | May                |

In [69]:
from enum import Enum

class PestType(str, Enum):
    green_stink_bug = "Green stink bug"
    pistachio_psylla = "Pistachio psylla"
    leaf_beetle = "Leaf beetle"

class Pest(BaseModel):
    type: PestType = Field(..., description="Type of pest, e.g., Green stink bug, Leaf beetle")
    RH_pest_min: float = Field(default=65, ge=0, le=100, description="Minimum relative humidity for pest activity")
    T_pest_min: float = Field(..., ge=0, le=50, description="Minimum temperature for pest activity")
    T_pest_max: float = Field(..., ge=0, le=50, description="Maximum temperature for pest activity")
    severity: float = Field(..., ge=0, le=1.0, description="Severity of pest [low=0.0, high=1.0]")
    treatment_difficulty: float = Field(..., ge=0, le=1.0, description="Difficulty to treat [low=0.0, high=1.0]")
    period_init: int = Field(..., ge=0, le=365, description="Month of the year when the pest can appear")
    period_end: int = Field(..., ge=0, le=365, description="Month of the year when the pest disappears")

    @field_validator('period_end')
    def check_period(cls, period_end, values):
        if 'period_init' in values and period_end < values['period_init']:
            raise ValueError("period_end must be after period_init")
        return period_end
    
# Green stink bug Class
class Green_stink_bug(Pest):
    type: PestType = PestType.green_stink_bug
    T_pest_min: float = 20
    T_pest_max: float = 35
    severity: float = np.random.uniform(0.3, 0.6)
    treatment_difficulty: float = np.random.uniform(0.6, 0.8)
    period_init: int = 8 # June
    period_end: int = 10 # October

# Pistachio psylla Class
class Pistachio_psylla(Pest):
    type: PestType = PestType.pistachio_psylla
    T_pest_min: float = 20
    T_pest_max: float = 35
    severity: float = np.random.uniform(0.6, 0.8)
    treatment_difficulty: float = np.random.uniform(0.6, 0.8)
    period_init: int = 4 # April
    period_end: int = 10 # October

# Leaf beetle Class
class Leaf_beetle(Pest):
    type: PestType = PestType.leaf_beetle
    T_pest_min: float = 20
    T_pest_max: float = 35
    severity: float = np.random.uniform(0.1, 0.3)
    treatment_difficulty: float = np.random.uniform(0.3, 0.6)
    period_init: int = 5 # May
    period_end: int = 5 # May

### Crop

<style scoped>
table {
  font-size: 16px;
  font-family: Verdana, sans-serif;
}
</style>

#### Rootstock

| **Characteristic**                 | **Rootstocks (in order of preference)**                                                                                  |
|------------------------------------|--------------------------------------------------------------------------------------------------------------------------|
| Cold Resistance                    | *P. Cornicabra (or P. Terebinthus)*, *P. Atlantica*                                                                       |
| Resistance to Verticillium         | *P. Integerrima (PGI)*, UCB-1                                                                                            |
| Salinity Resistance                | *P. Atlantica*, *P. Cornicabra*                                                                                          |
| Good Productivity in Poor Soils    | *P. Cornicabra*, *P. Vera*                                                                                               |
| High Yield                         | UCB-1, *P. Integerrima*, *P. Atlantica*, *P. Cornicabra*                                                                 |
| High Vigor (Trunk Diameter)        | *P. Atlantica*, *P. Cornicabra*, *P. Integerrima*, *P. Vera*                                                             |
| Recommended for Dryland Farming    | *P. Cornicabra*                                                                                                          |
| Recommended for Irrigated Farming  | *P. Atlantica*, UCB-1                                                                                                    |


In [70]:
from enum import Enum
from typing import Union

class RootstockType(str, Enum):
    P_Cornicabra = "P. Cornicabra"
    P_Atlantica = "P. Atlantica"
    UCB_1 = "UCB-1"
    P_Vera = "P. Vera"
    P_Integerrima = "P. Integerrima"

class Rootstock_P_Cornicabra(BaseModel):
    type: RootstockType = RootstockType.P_Cornicabra
    cold_resistance: float = Field(default=0.15, ge=0, le=1.0, description="Cold resistance level [low=0.0, high=1.0]")

class Rootstock_P_Atlantica(BaseModel):
    type: RootstockType = RootstockType.P_Atlantica
    vigour_increase: float = Field(default=0.15, ge=0, le=1.0, description="Vigor level [low=0.0, high=1.0]")
    cold_resistance: float = Field(default=0.15, ge=0, le=1.0, description="Cold resistance level [low=0.0, high=1.0]")
    salinity_resistance: float = Field(default=0.2, ge=0, le=1.0, description="Salinity resistance level [low=0.0, high=1.0]")

class Rootstock_UCB_1(BaseModel):
    type: RootstockType = RootstockType.UCB_1
    resistance_to_verticillium: float = Field(default=0.4, ge=0, le=1.0, description="Resistance to Verticillium level [low=0.0, high=1.0]")

class Rootstock_P_Vera(BaseModel):
    type: RootstockType = RootstockType.P_Vera
    vigour_increase: float = Field(default=0.15, ge=0, le=1.0, description="Vigor level [low=0.0, high=1.0]")

class Rootstock_P_Integerrima(BaseModel):
    type: RootstockType = RootstockType.P_Integerrima
    vigour_increase: float = Field(default=0.15, ge=0, le=1.0, description="Vigor level [low=0.0, high=1.0]")
    resistance_to_verticillium: float = Field(default=0.3, ge=0, le=1.0, description="Resistance to Verticillium level [low=0.0, high=1.0]")


# Variety Class
class Variety(BaseModel):
    vigour: float = Field(..., ge=0, le=1.0, description="Vigor level [low=0.0, high=1.0]")
    HU_optimal: float = Field(..., ge=0, le=5000, description="Optimal heat units for growth")
    T_base_HU: float = Field(..., ge=0, le=50, description="Base temperature for heat units")
    CH_optimal: float = Field(..., ge=0, le=5000, description="Optimal chill hours")
    T_base_CH: float = Field(..., ge=0, le=50, description="Base temperature for chill hours")
    alternate_bearing: float = Field(..., ge=0, le=1.0, description="Level of alternate bearing [low=0.0, high=1.0]")
    diseases: List[str] = Field(default_factory=list, description="List of diseases affecting the variety")
    pests: List[str] = Field(default=[*PestType] , description="List of pests affecting the variety")
    rootstock: Union[Rootstock_P_Cornicabra, Rootstock_P_Atlantica, Rootstock_UCB_1, Rootstock_P_Vera, Rootstock_P_Integerrima] = Field(default=Rootstock_P_Vera, description="Rootstock type")

# Crop Class
class Crop(BaseModel):
    init_age: int = Field(..., ge=10, le=100, description="Initial age of the crop [10-100 years]")
    variety: Variety

## Stress types

### Base stress

In biological systems, these processes are usually non-linear \cite{natalia2012}. For instance, when a disease spreads, it begins gradually and takes time to build up. However, as time passes, it spreads more rapidly and through more areas, making it an almost exponential process. Based on this foundation, we will create the following functions, each with the following common elements:

<ul>
<li>
    <strong>Trigger:</strong> <code>&phi;_{trigger}</code> is a function that is 1 when optimal conditions for the stress are accomplished and 0 when not.
</li>
    <li>
        <strong>Evolution function:</strong> It describes how the base stress evolves, due to direct causes described in table <a href="#tab:stress_causes">Table 1</a>. This ranges between 0 and 1 and depends on the relevant parameter. For example, in the case of cold-hour stress, it depends on the number of cold hours. Inside, there is a growth constant (<code>&alpha;_{stress}</code>): This regulates the speed at which stress develops. The range is from -10 to 10. -10 means exponential, 0 means linear and 10 means logarithmic (<code>"growth_rate"</code> in table <a href="#tab:initial-params-desc">Table 2</a>).
    </li>
    <li>
        <strong>Existing stress:</strong> Previous stress in the current year.
    </li>
    <li>
        <strong>Random component:</strong> This corresponds to a statistical distribution to generate randomness (<code>"random_component"</code> in table <a href="#tab:initial-params-desc">Table 2</a>).
    </li>
    <li>
        <strong>Corrective constants:</strong> These are values that help reduce stress. For example, high plant vigor or a high degree of expertise in the plantation can mitigate stress. This is modified by a lambda factor to simulate the importance of the associated stressor.
    </li>
    <li>
        <strong>Other stressors that accentuate this stress:</strong> Certain types of stress, when they reach a certain level, can trigger or exacerbate other types of stress. For example, poorly pruned branches can cause wounds (mechanical stress), which in turn make the plant more susceptible to pests (pest stress). <code>&phi;_{i}</code> is the trigger function of the stressor and <code>&lambda;_{i}</code> is the importance given to that stressor.
    </li>
</ul>


$$
S = \phi_{trigger} \cdot ( S_0 + 
\frac{1 - e^{-\alpha_{stress} |r_{stress}|}}{1 - e^{-\alpha_{stress}}} + 
\sum_{i=1}^{n_{stress}} \lambda_{i} \cdot \phi_{i} \cdot S_{i} - 
\sum_{i=1}^{n_{corrective}} \beta_{i} + \epsilon)
$$

$
\phi_{i} = \begin{cases}
    1 & S_i > S_{threshold\_i} \\
    0 & S_i \le S_{threshold\_i}
\end{cases}
$

In [ ]:
class Stress(BaseModel):
    stress: float = 0.0
    ## Exponential function params
    growth_rate: float = Field(..., ge=-10, le=10, description="Growth rate [-10 to 10]")
    ## Other stressors params
    stressor_weight: List[float] = Field(..., description="Weights of stressors [0.0-1.0]")
    causal_stress_init: List[float] = Field(..., description="Initial causal stress weights [0.0-1.0]")
    other_stressors: List[float] = Field(..., description="Other stressors [0.0-1.0]")
    ## Corrective constants params
    corrective_constants: List[float] = Field(..., description="Corrective constants [0.0-1.0]")
    ## Random component
    random_component: float = Field(..., description="Random component")

    # Allow arbitrary types (ndarray in this case)
    class Config:
        arbitrary_types_allowed = True

    @model_validator(mode="after")
    def check_stress_weights(self):
        if len(self.stressor_weight) != len(self.causal_stress_init):
            raise ValueError("stressor_weight and causal_stress_init must have the same length")
        if len(self.stressor_weight) != len(self.other_stressors):
            raise ValueError("stressor_weight and other_stressors must have the same length")
        # Check that List[float] are between 0 and 1
        if not all(0 <= weight <= 1 for weight in self.stressor_weight):
            raise ValueError("stressor_weight must be between 0 and 1")
        if not all(0 <= causal_stress <= 1 for causal_stress in self.causal_stress_init):
            raise ValueError("causal_stress_init must be between 0 and 1")
        if not all(0 <= stressor <= 1 for stressor in self.other_stressors):
            raise ValueError("other_stressors must be between 0 and 1")
        if not all(0 <= corrective_constant <= 1 for corrective_constant in self.corrective_constants):
            raise ValueError("corrective_constants must be between 0 and 1")
        return self
    
    @property
    def _trigger(self):
        print("Implement this method in corresponding stress class...")
        raise NotImplementedError

    @property
    def _magnitude_of_stress(self):
        print("Implement this method in corresponding stress class...")
        raise NotImplementedError

    @property
    def _evolution_function(self):
        magnitude = self._calculate_magnitude_of_stress(self)
        return (1 - np.exp(-self.growth_rate) * np.abs(magnitude)) / (1 - np.exp(-self.growth_rate))

    @property
    def _other_stressors(self):
        activation_function = [1 if stressor > causal_stress else 0 for stressor, causal_stress in zip(self.other_stressors, self.causal_stress_init)]
        return np.sum(np.multiply(self.stressor_weight, activation_function))

    @property
    def _corrective_constants(self):
        return np.sum(self.corrective_constants)

    def calculate_stress(self):
        if self._trigger:
            self.stress += self._evolution_function() \
                + self._other_stressors() \
                - self._corrective_constants() \
                + self.random_component
        return self.stress


### Chill hours stress

$$
    S_{chill\_hours}(CH) = \begin{cases}
    \frac{1 - e^{-\alpha_{cold} |r_{cold}|}}{1 - e^{-\alpha_{cold}}} + \epsilon & CH < CH_{optimal}\\
    0 & CH \ge CH_{optimal}
\end{cases}
$$

$CH_{optimal} =$ are the optimal chill hours for the corresponding pistachio variety.

$r_{cold} = 0 \ge \frac{CH}{CH_{optimal}} > 1$

$\epsilon \sim N(\mu, \sigma)$

In [72]:
class ChillHoursStress(Stress):
    # Stress input params
    CH: float = Field(..., ge=0, le=5000, description="Chill hours")
    # Stress specific local params
    CH_optimal: float = Field(..., ge=0, le=5000, description="Optimal chill hours")

    @property
    def _trigger(self):
        return self.CH < self.CH_optimal

    @property
    def _magnitude_of_stress(self):
        return min(self.CH / self.CH_optimal, 1)

### Heat units stress

$$
    S_{chill\_hours}(CH) = \begin{cases}
    \frac{1 - e^{-\alpha_{cold} |r_{cold}|}}{1 - e^{-\alpha_{cold}}} + \epsilon & CH < CH_{optimal}\\
    0 & CH \ge CH_{optimal}
\end{cases}
$$

$HU_{optimal} =$ are the optimal heat units for the corresponding pistachio variety.

$r_{heat} = 0 \ge \frac{HU}{HU_{optimal}} > 1$

$\epsilon \sim N(\mu, \sigma)$

In [73]:
class HeatUnitsStress(Stress):
    # Stress input params
    HU: float = Field(..., ge=0, le=5000, description="Heat units")
    # Stress specific local params
    HU_optimal: float = Field(..., ge=0, le=5000, description="Optimal heat units")

    @property
    def _trigger(self):
        return self.HU < self.HU_optimal

    @property
    def _magnitude_of_stress(self):
        return min(self.HU / self.HU_optimal, 1)

### Temperature stress

$$
S_{temp}(H_{T\_ext}) = \begin{cases}
    S_{temp\_0} + \frac{1 - e^{-\alpha_{temp} |r_{temp}|}}{1 - e^{-\alpha_{temp}}} - \sum_{i=1}^{n\_correct} \beta_{i} + \epsilon & H_{T\_ext} > 0\\
    0 & H_{T\_ext} = 0
\end{cases}
$$

$H_{T\_ext} = hours(T \ge T_{max} \vee T_{min} \ge T)$

$r = 0 \ge \frac{H_{T\_ext}}{H_{T\_ext\_max}} \ge 1$

% $H_{T\_ext\_max} =$ 1 month (730 hours) of extreme temperatures between $T_{min}$ and $T_{max}$.

$n_{correct} =$ \{cold\_resistance\}

$\epsilon \sim N(\mu, \sigma)$

In [74]:
class TemperatureStress(Stress):
    # Stress input params
    hours_under_extreme_temperature: int = Field(..., ge=0, description="Hours under extreme temperature")
    # Stress specific local params
    max_hours_under_extreme_temperature: int = Field(..., ge=0, le=24, description="Max hours under extreme temperature")
    
    @property
    def _trigger(self):
        return self.hours_under_extreme_temperature > 0

    @property
    def _magnitude_of_stress(self):
        return min(self.hours_under_extreme_temperature / self.max_hours_under_extreme_temperature, 1)

### Hydric stress

$$
S_{hydric}(PP) = \begin{cases}
    S_{hydric\_0} + \frac{1 - e^{-\alpha_{hydric} |r_{hydric}|}}{1 - e^{-\alpha_{hydric}}} - \sum_{i=1}^{n\_correct} \beta_{i} + \epsilon & PP < PP_{min} \vee PP > PP_{max}\\
    S_{hydric\_0} & PP_{min} \ge PP \ge PP_{max}
\end{cases}
$$

$r_{hydric} = 0 \ge \frac{PP - PP_{min}}{PP_{max} - PP_{min}} \ge 1$

$n_{correct} =$  \{irrigation, drainage\}

In [75]:
class HydricStress(Stress):
    # Stress input params
    PP: float = Field(..., ge=0, le=100, description="Precipitation level")
    # Stress specific local params
    PP_min: float = Field(..., ge=0, le=100, description="Minimum precipitation level")
    PP_max: float = Field(..., ge=0, le=100, description="Maximum precipitation level")

    @property
    def _trigger(self):
        return self.PP < self.PP_min or self.PP > self.PP_max

    @property
    def _calculate_magnitude_of_stress(self):
        # TODO: PENSARR BIEEEEN!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        if self.PP < self.PP_min:
            return self.PP / self.PP_min
        else:
            return self.PP / self.PP_max

### Mechanical stress

$$
\small
S_{mechanical} = S_{mechanical\_0} + 1 - M - \sum_{i=1}^{n_{correct}} \beta_{i}
$$

$M \sim N(\mu, \sigma)$

$n_{correct} =$ \{vigour, machinery, experience\}

In [76]:
class MechanicalStress(Stress):
    # Stress specific local params
    farming_activity: np.ndarray = Field(..., description="Farming activity")

    @property
    def _trigger(self):
        return True

    @property
    def _evolution_function(self):
        return np.random.choice(self.farming_activity)

### Nutritional stress

$$
S_{nutri} = \begin{cases}
 S_{nutri\_0} + 1 - N + \sum_{i=1}^{n\_stress} \lambda_{i} \cdot \phi_{i} \cdot S_{i} - \sum_{i=1}^{n\_correct} \beta_{i} & \text{fertilisation}\\
 S_{nutri\_0} & \text{no fertilisation}
\end{cases}
$$

$N \sim N(\mu, \sigma)$ represents the success of fertilisation (fert) activity between 0 and 1.

$n\_estres =$ \{hydric\}

$
\phi_{i} = \begin{cases} 
    1 & S\_hydric > 0.5 \\
    0 & S\_hydric \le 0.5 \\
\end{cases}
$

$n_{correct} =$ \{experience, salinity\_resistance\}

$epsilon \sim N(\mu, \sigma)$

In [77]:
class NutritionalStress(Stress):
    # Stress specific local params
    fertilisation: np.ndarray = Field(..., description="Fertilisation activity")

    @property
    def _trigger(self):
        return True

    @property
    def _evolution_function(self):
        return np.random.choice(self.fertilisation)

### Pest and disease stress

$$
S_{pest}(H_{RH\_pest}, H_{T\_pest}) = \begin{cases}
    S_{pest\_0} + \frac{1 - e^{-\alpha_{pest} |r_{pest}|}}{1 - e^{-\alpha_{pest}}} + \sum_{i=1}^{n_{stress}} \lambda_{i} \cdot \phi_{i} \cdot S_{i} - \sum_{i=1}^{n_{correct}} \beta_{i} + \epsilon & H_{RH\_pest} > 0 \wedge H_{T\_pest} > 0 \\
    S_{pest\_0} & H_{RH\_pest} = 0 \vee H_{T\_pest} = 0
\end{cases}\\
$$

$H_{RH\_pest} = hours(RH \ge RH\_pest\_min)\ \&\ H_{T\_pest} = hours(T_\_pest\_min \le T \le T_\_pest\_max)$ These are the hours of exposition to favourable relative humidity and temperature of pest.

$n_{stress} =$ \{mechanical, heat\_units\}

$n_{correct} =$ {treatment, vigour, vigour\_increase, experience}

$\alpha_{pest}$ is the severity of the pest, that represents the growth speed of the pest.

$r_{pest} = 0 \ge \frac{H_{RH\_pest}}{H_{RH\_pest\_max}} \ge 1$

$\epsilon \sim N(\mu, \sigma)$

In [78]:
class PestDiseaseStress(Stress):
    # Stress input params
    hours_under_optimal_temperature: int = Field(..., description="Hours under optimal temperature in a period")
    hours_under_optimal_relative_humidity: int = Field(..., description="Hours under optimal relative humidity in a period")
    # Stress specific local params
    max_hours_under_optimal_temperature: int = Field(..., ge=0, description="Max hours under optimal temperature")
    max_hours_under_optimal_relative_humidity: int = Field(..., ge=0, description="Max hours under optimal relative humidity")

    @property
    def _trigger(self):
        return self.hours_under_optimal_temperature > 0 and self.hours_under_optimal_relative_humidity > 0

    @property
    def _magnitude_of_stress(self):
        return min(self.hours_under_optimal_temperature / self.max_hours_under_optimal_temperature, 1) \
            + min(self.hours_under_optimal_relative_humidity / self.max_hours_under_optimal_relative_humidity, 1)

### Total stress

Equation shows the accumulated total stress that is calculated as the weighted average of each corresponding stress and the residual stress, where each $\lambda_{i}$ represents the weighted importance assigned to each one, $n_{stress} \in$ {chill hours, temperature, heat units, water, mechanical, nutritional, pest, disease}, $\lambda_{prev}$ is the weighted importance assigned to the previous residual stress and $S_{prev}$ is the previous residual stress of last productive years. $S_{total}$ must always be between 0 and 1.

$$
S_{total} = \lambda_{prev} \cdot S_{prev} + \frac{\sum_{i=1}^{n\_stress} \lambda_{i} \cdot S_{i}}{\sum_{i=1}^{n\_stress} \lambda_{i}}
$$

In [79]:
class TotalStress(BaseModel):
    total_stress: float = Field(..., ge=0, le=1.0, description="Total stress level [0.0-1.0]")
    # Stress specific local params
    stressors: List[Tuple[float, float]] = Field(..., description="List of tuples where each tuple contains (stress_weight, stress_level)")
    previous_stress_weight: float = Field(..., ge=0, le=1.0, description="Previous stress weight [0.0-1.0]")
    previous_stress: float = Field(..., ge=0, le=1.0, description="Previous stress level [0.0-1.0]")

    def _calculate_magnitude_of_stress(self):
        effective_previous_stress = self.previous_stress_weight * self.previous_stress
        weighted_sum = sum([stress_weight * stress for stress_weight, stress in self.stressors]) / sum([stress_weight for stress_weight, _ in self.stressors])
        self.total_stress =  effective_previous_stress + weighted_sum
        return self.total_stress

## Stress-based estimated yield equation

The following equation calculates the approximated yield per tree in a productive year, where γpollination
is the degree of female tree pollination, γbearing is the alternate bearing suffered -depends on
“on”-“off” cycle-, γage is the increase of yield that depends on the tree’s age, Stotal is the total
accumulated stress, Ybase is a baseline yield determined by extracted knowledge and ϵ is a
probabilistic distribution that adds randomness. Ybase depends on whether the tree is grown
under rainfed or irrigated conditions

$$
Y = \gamma_{pollination} \cdot (\gamma_{bearing} \cdot (1 - S_{total}) \cdot Y_{base} + \gamma_{age}) + \epsilon\\
$$

$\gamma_{age}$ represents that every two years production raises 1 kilograms in case of rainfed and 1.5 kilograms in case of irrigated.

$$
\small
\gamma_{age} = \begin{cases}
    1\ kilogram & \ge 10\ years\ old \wedge rainfed \\
    1.5\ kilograms & \ge 10\ years\ old \wedge irrigated
\end{cases}
$$

$\epsilon \sim N(\mu, \sigma)$\\

$\gamma_{bearing}$ represents the intensity of the alternate bearing and depends on the last year production and the pistachio variety. It should be between 50\% and 80\% less than the last year production, depending on the stress levels.

$$
\small
\gamma_{bearing} = \begin{cases}
     0.2 \le \epsilon_{bearing} - 0.2 \cdot S_{total} \le 0.5 & ``off'' year\\
     1 & ``on'' year
\end{cases}
$$

METER EL parametro alternate\_bearing!!!!!

$\epsilon_{bearing} \sim U(0.3, 0.5)$\\

$\gamma_{pollination}$ represents the percentage of pollination of male trees to female ones in April. It ranges between 0 and 1.

$$
\small
\gamma_{pollination} = \begin{cases}
    1 - \frac{1 - e^{-\alpha_{pollination} |r_{pollination}|}}{1 - e^{-\alpha_{pollination}}} - \lambda_{CH} \cdot S_{CH} + \epsilon & avg(RH) > 85\% \vee PP > 50 mm\\
    1 & avg(RH) < 85\% \wedge PP \le 50 mm
\end{cases}
$$

$
r_{pollination} = 0 \ge \frac{avg(RH) - avg(RH_{min})}{avg(RH_{max}) - avg(RH_{min})} \cdot \frac{PP - PP_{min}}{PP_{max} - PP_{min}} > 1
$\\

$Y_{base}$ is the base production of the registered average according to knowledge. We have seen that is around 8 kilograms in rainfed regime and around 15 kilograms in irrigation regime.

$$
\small
Y_{base} = \begin{cases}
    8\ kilograms & \ge 10\ years\ old \wedge rainfed \\
    15\ kilograms & \ge 10\ years\ old \wedge irrigated
\end{cases}
$$

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal
import numpy as np

class EstimatedYield(BaseModel):
    # Input parameters
    age: int = Field(..., ge=10, description="Age of the tree in years")
    irrigation: Literal["rainfed", "irrigated"] = Field(..., description="Irrigation type (rainfed or irrigated)")
    alternate_bearing: float = Field(..., ge=0.5, le=0.8, description="Intensity of alternate bearing (0.5 to 0.8)")
    total_stress: float = Field(..., ge=0, le=1, description="Total accumulated stress (0 to 1)")
    avg_rh_april: float = Field(..., ge=0, le=100, description="Average relative humidity in April (%)")
    pp_april: float = Field(..., ge=0, description="Precipitation in April (mm)")
    alpha_pollination: float = Field(..., ge=0, description="Pollination growth rate parameter")
    lambda_ch: float = Field(..., ge=0, le=1, description="Weight of chill hours stress in pollination")
    s_ch: float = Field(..., ge=0, le=1, description="Chill hours stress (0 to 1)")
    random_component: float = Field(..., description="Random component")
    y_base_rainfed: float = Field(8, description="Base yield for rainfed trees")
    y_base_irrigated: float = Field(15, description="Base yield for irrigated trees")
    max_pollination_rh: float = Field(85, description="Maximum relative humidity for pollination")
    max_pollination_pp: float = Field(50, description="Maximum precipitation for pollination")

    # Allow arbitrary types (ndarray in this case)
    class Config:
        arbitrary_types_allowed = True

    @property
    def is_on_year(self) -> bool:
        """Check if the tree is "on" year. In this case, if the age is even."""
        return self.age % 2 == 0

    # Computed properties
    @property
    def _gamma_age(self) -> float:
        """Compute gamma_age based on the age and irrigation type."""
        if self.age >= 10:
            return 1.5 if self.irrigation == "irrigated" else 1.0
        return 0.0

    @property
    def _gamma_bearing(self) -> float:
        """Compute gamma_bearing based on alternate bearing and stress levels."""
        epsilon_bearing = np.random.uniform(0.3, 0.5)
        if not self.is_on_year:
            return max(0.2, min(epsilon_bearing - 0.2 * self.total_stress, 0.5))
        return 1.0

    @property
    def _gamma_pollination(self) -> float:
        # TODO: QUIZAS DERIVAR DE LA CLASE Stress PORQUE LA FORMULA ES LA MISMA
        """Compute gamma_pollination based on humidity, precipitation, and stress."""
        # if self.avg_rh_april > self.max_pollination_rh or self.pp_april > self.max_pollination_pp:
        #     r_pollination = max(0, min(
        #         ((self.avg_rh_april - 60) / (self.max_pollination_rh - 60)) * ((self.pp_april - 10) / (self.max_pollination_pp - 10)), 1))
        #     exponential_term = (
        #         1 - np.exp(-self.alpha_pollination * abs(r_pollination))
        #     ) / (1 - np.exp(-self.alpha_pollination))
        #     return 1 - exponential_term - self.lambda_ch * self.s_ch + self.random_component
        # return 1.0
        return 1.0

    @property
    def _y_base(self) -> float:
        """Compute Y_base based on age and irrigation type."""
        if self.age >= 10:
            return self.y_base_rainfed if self.irrigation == "rainfed" else self.y_base_irrigated
        return 0.0

    def calculate_yield(self) -> float:
        """Calculate the estimated yield per tree."""
        random_component = self.random_component
        return (
            self._gamma_pollination *
            (self._gamma_bearing * (1 - self.total_stress) * self._y_base + self._gamma_age) +
            random_component
        )

## Simulation parameters

<style scoped>
table {
  font-size: 16px;
  font-family: Verdana, sans-serif;
}
</style>

In this case, we are going to be using the **Kerman variety** of pistachio, which is the most common variety in Spain. It is usually used with the rootstock **Pistacia atlantica**. The following table shows the initial parameters for the simulation.

| Parameter | Description | Value | Units |
| --- | --- | --- | --- |
| $CH\_optimal$ | Optimal chill hours for Kerman variety | 1000 | hours |
| $HU\_optimal$ | Optimal heat units for Kerman variety | 3500 | units |
| $T\_min$ | Minimum temperature for the Kerman variety | -30 | ºC |
| $T\_max$ | Maximum temperature for the Kerman variety | 50 | ºC |
| $T\_base\_CH$ | Base temperature for chill hours calculation | < 7 | ºC |
| $T\_base\_HU$ | Base temperature for heat units calculation | > 10 | ºC |
| Vigour | Plant vigour | Medium | - |
| alternate_bearing | Alternate bearing | High | - |
| stress_weight | Weight of corresponding stress | $stress\_weight_{[Botryosphaeria, Alternaria]}$ = 1.0 | - |
| rootstock | Rootstock | Pistacia atlantica | - |
| init_age | Initial age of the crop | [10, 100] | years |


In [166]:
pest_stress = PestDiseaseStress(
    hours_under_optimal_temperature=10,
    hours_under_optimal_relative_humidity=10,
    stress=0.5,
    stressor_weight=[0.5, 0.5],
    causal_stress_init=[0.5, 0.5],
    other_stressors=[0.5, 0.5],
    corrective_constants=[0.5, 0.5],
    random_component=0.5,
    growth_rate=1.0,
    max_hours_under_optimal_temperature=24,
    max_hours_under_optimal_relative_humidity=24
)

np.random.normal(0.5, 0.1)

0.4711448887358477